In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[30,15]

In [ ]:
df = pd.read_csv("Dataset1.csv")

In [ ]:
df.drop(columns=['CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'], inplace=True)
df.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df.dropna(subset=['BOROUGH'], inplace=True)
df = df[df['LATITUDE']!=0]
df = df[df['LONGITUDE']!=0]

In [ ]:
df1 = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != "Unspecified"]
df1 = df1.sort_values(by=['CONTRIBUTING FACTOR VEHICLE 1'])
df_temp1 = df1.groupby(by=['CONTRIBUTING FACTOR VEHICLE 1'])
s3 = df_temp1.size()

df2 = df[df['CONTRIBUTING FACTOR VEHICLE 2'] != "Unspecified"]
df2 = df2.sort_values(by=['CONTRIBUTING FACTOR VEHICLE 2'])
df_temp2 = df2.groupby(by=['CONTRIBUTING FACTOR VEHICLE 2'])
s4 = df_temp2.size()

df_equal = df[df['CONTRIBUTING FACTOR VEHICLE 1'] == df['CONTRIBUTING FACTOR VEHICLE 2']]
df_equal = df_equal[df_equal['CONTRIBUTING FACTOR VEHICLE 1'] != "Unspecified"]
df_equal = df_equal.sort_values(by=['CONTRIBUTING FACTOR VEHICLE 1'])
df_equal_temp = df_equal.groupby(by=['CONTRIBUTING FACTOR VEHICLE 1'])
s5 = df_equal_temp.size()

In [ ]:
s_total_ = s3.add(s4, fill_value=0)
s_total = s_total_.sub(s5, fill_value=0)
bars = s_total.index
x_pos = range(len(s_total))
plt.bar(x_pos, s_total.values, width=0.8)
plt.xlabel("Cause of accident", fontsize=15)
plt.ylabel("Accidents", fontsize=15)
plt.title("Accidents by type in New York City", fontsize=15)
plt.xticks(x_pos, bars, rotation = 85, fontsize=15)
plt.yticks(fontsize=15)
for i in range(len(s3)):
    plt.text(i, s_total.astype('int32')[i], s_total.astype('int32')[i], ha='center', bbox = dict(facecolor = 'white', alpha =.8), fontsize=15)
plt.show()

In [ ]:
groups = df.groupby(by=['BOROUGH'])
for name, group in groups:
    df1_temp = group[group['CONTRIBUTING FACTOR VEHICLE 1'] != "Unspecified"]
    df1_temp = df1_temp.sort_values(by=['CONTRIBUTING FACTOR VEHICLE 1'])
    df_group1 = df1_temp.groupby(by=['CONTRIBUTING FACTOR VEHICLE 1'])
    s3 = df_group1.size()

    df2_temp = group[group['CONTRIBUTING FACTOR VEHICLE 2'] != "Unspecified"]
    df2_temp = df2_temp.sort_values(by=['CONTRIBUTING FACTOR VEHICLE 2'])
    df_group2 = df2_temp.groupby(by=['CONTRIBUTING FACTOR VEHICLE 2'])
    s4= df_group2.size()

    df_equal = group[group['CONTRIBUTING FACTOR VEHICLE 1'] == group['CONTRIBUTING FACTOR VEHICLE 2']]
    df_equal = df_equal[df_equal['CONTRIBUTING FACTOR VEHICLE 1'] != "Unspecified"]
    df_equal = df_equal.sort_values(by=['CONTRIBUTING FACTOR VEHICLE 1'])
    df_equal_temp = df_equal.groupby(by=['CONTRIBUTING FACTOR VEHICLE 1'])
    s5 = df_equal_temp.size()

    s_total_ = s3.add(s4, fill_value=0)
    s_total = s_total_.sub(s5, fill_value=0)

    print(name)
    bars = s_total.index
    x_pos = range(len(s_total))
    plt.bar(x_pos, s_total.values, width=0.8)
    plt.xlabel("Cause of accident", fontsize=15)
    plt.ylabel("Accidents", fontsize=15)
    plt.title(f"Accidents by type in {name}", fontsize=15)
    plt.xticks(x_pos, bars, rotation = 85, fontsize=15)
    plt.yticks(fontsize=15)
    for i in range(len(s3)):
        plt.text(i, s_total.astype('int32')[i], s_total.astype('int32')[i], ha='center', bbox = dict(facecolor = 'white', alpha =.8), fontsize=15)
    plt.show()